## Zaawansowane Metody Inteligencji Obliczeniowej
# Zadanie domowe 5 (10 pkt.)
### Prowadzący: Michał Kempka, Marek Wydmuch
### Autor: twoje imię i nazwisko + numer indeksu, grupa

## Zadanie 5

Świat jest prostokątem, o wymiarach **N** wierszy na **M** kolumn. Każde pole może być jamą (1), wyjście (2) lub puste (0). Celem Wumpusa jest najszybsze dotarcie do wyjścia.
Wumpus porusza się w świecie wykonując ruchy **UP**, **DOWN**, **RIGHT**, **LEFT**.
Świat nie ma ścian i jest cykliczny, to znaczy, że wykonując ruch **UP** z pola w pierwszym (górnym) wierszu,
Wumpus (zwykle) dorze do pola w ostatnim (dolnym) wierszu; analogicznie w przypadku ruchw poziomych.
Wumpus jest otumaniony, dlatego jest niepewny swoich akcji.
Wykonując ruch z prawdopodobieńśtwem **p** (np. 0.8) dotrze do docelowego pola, ale z prawd. (1-**p**)/4 (np. 0.05) znajdzie się na jednym z czterech pól sąsiadujących z polem docelowym.

W świecie Wumpusa występują jamy.
Dla otumanionego Wumpusa nie są one jednak groźne, a stanowią dla niego ważne punkty orientacyjne.
Jeśli Wumpus stanie na polu, na którym jest jama, Wumpus odnotowuje ten fakt z prawdopodobieństwem **pj** (np. 0.7).
Niestety, ze względu na otumanienie, czasem (z prawd. **pn** (np. 0.1)) wydaje mu się, że wpadł do jamy, gdy tymczasem nic takiego nie miało miejsca.

Wumpus posiada mapę świata, ale nie ma pojęcia, gdzie się znajduje. Pomóż mu w jak najmniejszej liczbie ruchów dotrzeć do pola oznaczonego jako Wyjście.
Celem minimalizacji jest dojść Wumpusem w **średnio** jak najmniejszej liczbie kroków przy niedoskonałej wiedzy i niedoskonałej motoryce.
#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod) o ile nie napisano gdzieś inaczej.

## Repozytorium
Do wykonania zadania potrzebny jest pakiet **misio**, który zainstaluje środowisko i wszystkie dependencje.

In [ ]:
!pip3 install git+https://github.com/mihahauke/misio_labs

# lub:
# !git clone https://github.com/mihahauke/misio_labs
# !cd misio
# !sudo pip3 install .



## Przykładowy świat
Do oceny rozwiązań zostaną użyte dwa pliki 2015.in i 2016.in, dostarczone razem z tym notebookiem. Pliki te zawierają wiele map. By wczytać dowolny świat z plików wejściowych możemy użyć poniższego kodu.

In [ ]:
from misio.lost_wumpus.util import load_input_file

worlds = load_input_file("2015.in")
# każdy świat opisany jest przez mapę i odpowiednie prawdopodobieńśtwa
m, p, pj, pn = worlds[0]
print(m, p, pj, pn)


## Losowy agent
Poniżej znajduje się implementacja prostego, losowego agenta ( dziedziczącego po misio.lost_wumpus.agents.AgentStub) implementującego następujące funkcje:
**move** zwracającą losowy ruch, który ma wykonać agent.
* **sense**(bool) - aktualizacja wiedzy na podstawie wykrycia (lub nie) jamy - w przypadku losowego agenta nic się nie dzieje
* **move**() - metoda zwracająca jeden (losowy) z ruchów z klasy misio.lost_wumpus._wumpus.Action
* **reset**() - zresetowanie wiedzy agenta na potrzeby wielu testów na tej samej mapie - znowu, w przypadku agenta losowego nie jest to potrzebne

In [ ]:
import numpy as np
from misio.lost_wumpus.agents import AgentStub
from misio.lost_wumpus._wumpus import Action

class RandomAgent(AgentStub):
    def __init__(self, m, p, pj, pn):
        super(RandomAgent, self).__init__(m, p, pj, pn)

    def move(self):
        # zrób losowy ruch
        return np.random.choice(Action)

    def sense(self, sensory_input: bool):
        # nie aktualizuj wiedzy bo i tak robione są losowe ruchy
        pass

    def reset(self):
        # nie ma co resetować bo agent jest losowy i nie utrzymuje żadnej wiedzy
        pass

## Odpalenie agenta
Poniższy kod prezentuje jak przetestować agenta na danej mapie 10 razy.

In [ ]:
import tqdm
from misio.lost_wumpus import LostWumpusGame
from misio.lost_wumpus.testing import default_steps_constraint


def test_world(world, agent_class, n=10):
    # mapa i odpowiednie prawdopodobieństwa podane w opisie zadania
    m, p, pj, pn = world
    agent = agent_class(m, p, pj, pn)
    # ustawia maksymalną liczbę kroków w zależności od wielkości mapy
    max_moves = default_steps_constraint(m)
    # tworzy grę
    game = LostWumpusGame(m, p, pj, pn, max_moves=max_moves)

    run_scores = []
    for _ in tqdm.trange(n, leave=False):
        # reset agenta (jego wiedzy)
        agent.reset()
        # reset gry
        game.reset()
        # póki gra się nie skończy agent działą
        while not game.finished:
            # wyczuwanie czy jama zostałą wykryta
            agent.sense(game.sensory_output)
            # wybieranie ruchu
            move = agent.move()
            # powiadomienie środowiska o ruchu
            game.apply_move(move)
        # wynik, który chcemy minimalizować
        number_of_moves_performed = game.moves
        run_scores.append(number_of_moves_performed)
    return run_scores

# ładuje pierwszą mapa z brzegu
world = load_input_file("2015.in")[0]
run_scores = test_world(world, RandomAgent,n=200)
print("Average number of moves: {:0.2f}".format(np.mean(run_scores)))

## Testowanie
Do oceny rozwiązań zostaną użyte dwa pliki 2015.in i 2016.in. By przetestować agenta na każdym świecie można użyć funkcji **test_locally** z pakietu misio.

### Punktacja (10 punktów)
Każdy świat zostanie przetestowany conajmniej 100 razy, a wyniki uśrednione.
Następnie liczba punktów zostanie policzona wedle następującego wzoru:

max(0,min(10,(7000-(**score2015** +**score2016**)/2)/(7000-4600)*10))

gdzie **score2015** i **score2016** to uśrednione wyniki z odpowiednich plików.

Jak widać, uzyskanie  **4600** kroków daje maksymalny wynik 10 punktów, każda wartość poniżej **7000** kroków da zero punktów, a wszelkie wartości pomiędzy są interpolowane liniowo (od 0 do 10).

> Uwaga: W przypadku jakiegokolwiek wyjątku rzuconego w wyniku błędnego działania agenta, na którymkolwiek teście ostateczne punkty wyniosą 0 niezależnie od wyników na innych światach.

In [ ]:
# Wczytywanie światów
worlds_2015 = load_input_file("2015.in")
worlds_2016 = load_input_file("2016.in")
# Testowanie:
from misio.lost_wumpus.testing import test_locally
# przetestuje agenta 5 razy na każdym świecie
# 5 testów może dawać niestabilne wyniki, lecz jest szybsze na potrzeby demonstracji
# dodatkowo, należy pamiętać, że lepszy wynik będzie testowany krócej
# losowy agent jest wyjątkowo zły i jego testowanie zamie sporo czasu
score2015 = test_locally("2015.in", RandomAgent, n=5)
score2016 = test_locally("2016.in", RandomAgent, n=5)

print("Average numbers of moves: {:0.1f} & {:0.1f}".format(score2015, score2016))
print("Points: {:0.1f}".format(points))

## Rozwiązanie (tutaj powinno znajdować się Twoje rozwiązanie)
Rozwiązanie musi dziedziczyć po klasie **misio.lost_wumpus.agents.AgentStub** i implementować następujące metody (może też implementować dodatkowe metody):
* sense(bool) - aktualizacja wiedzy na podstawie wykrycia (lub nie) jamy
* move() - metoda zwracająca jeden z ruchów z klasy misio.lost_wumpus._wumpus.Action
* reset() - zresetowanie wiedzy agenta na potrzeby wielu testów na tej samej mapie
> Uwaga: Nie twórz dodatkowych komórek na potrzeby rozwiązania i **nie** dopisuj do poniższej komórki żadnego kodu poza **importami** i dowolnymi **nowymi metodami** klasy. Sprawdzarka wczyta tylko tę klasę i ewentualne importy.

In [ ]:
from misio.lost_wumpus.agents import AgentStub
from misio.lost_wumpus._wumpus import Action
class LostWumpusAgent(AgentStub):
    def __init__(self, m: np.ndarray, p: float, pj: float, pn: float):
        super(LostWumpusAgent, self).__init__(m, p, pj, pn)
        # TODO dowolna inicjalizacja tutaj
    def sense(self, sensory_input: bool):
        raise NotImplementedError()

    def move(self):
        # TODO wybranie ruchu, zwróć jedną z dozwolonych akcji
        raise NotImplementedError()

    def reset(self):
        # TODO zresetowanie wiedzy na potrzeby kolejnego testu na tej samej mapie
        raise NotImplementedError()

In [ ]:
from misio.lost_wumpus.testing import test_locally
score2015 = test_locally("2015.in", LostWumpusAgent, n=5)
score2016 = test_locally("2016.in", LostWumpusAgent, n=5)

points = max(0, min(10, (7000 - (score2015 +score2016)/2)/(7000 - 4600) * 10))
print("Average numbers of moves: {:0.1f} & {:0.1f}".format(score2015, score2016))
print("Points: {:0.1f}".format(points))